In [1]:
import pandas as pd
import numpy as np
import sklearn 
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import KBinsDiscretizer

from sklearn.pipeline import Pipeline
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.decomposition import NMF
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

import pickle

Import the clubs data csv. Group the clubs and join all genres into list in a single column.

In [2]:
event_data=pd.read_csv('event_data.csv')

In [3]:
ev= event_data.groupby('clubs')['genres'].apply(','.join).reset_index()
ev

,clubs,genres
0,40seconds Club,"dub techno,dance pop,dub techno,chanson,compos..."
1,Abspannwerk Buchhändlerhof,"east coast hip hop,electro-pop francais,detroi..."
2,Adagio Berlin,"australian dance,armenian pop,turkish edm,nouv..."
3,Arena Berlin,"happy hardcore,eurovision,melodic techno,anado..."
4,Astra Kulturhaus,"deep tropical house,arab electronic,mandible,g..."
...,...,...
56,Watergate (Berl3 F),"turkish edm,acid techno,intelligent dance musi..."
57,Weekend Club,"electro-pop francais,bergen indie,lo-fi chill,..."
58,Wilde Renate,"ninja,pop dance,gabba,experimental techno,pres..."
59,YAAM (club),"modern reggae,classical,german dance,classic f..."


In [42]:
clubs=pd.DataFrame(ev['clubs'])
clubs

,clubs
0,40seconds Club
1,Abspannwerk Buchhändlerhof
2,Adagio Berlin
3,Arena Berlin
4,Astra Kulturhaus
...,...
56,Watergate (Berl3 F)
57,Weekend Club
58,Wilde Renate
59,YAAM (club)


Use Getdummies to transform the list in 'genres' column into a OneHotEncoded format for use in the model.

In [5]:
ev_ohe= ev.genres.str.get_dummies(',')
ev_ohe= clubs.join(ev_ohe)
ev_ohe

,clubs,acid techno,afro house,alternative dance,alternative hip hop,anadolu rock,arab electronic,arabesk,armenian hip hop,armenian pop,...,turkish folk,turkish hip hop,turkish pop,turkish rock,turkish trap,uk dancehall,urbano espanol,vapor twitch,violin,york indie
0,40seconds Club,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Abspannwerk Buchhändlerhof,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,Adagio Berlin,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Arena Berlin,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,Astra Kulturhaus,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,Watergate (Berl3 F),1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57,Weekend Club,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58,Wilde Renate,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59,YAAM (club),0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Extract all the column names to create a list of all genres

In [6]:
genrelist = ev_ohe.columns
genrelist

Index(['clubs', 'acid techno', 'afro house', 'alternative dance',
       'alternative hip hop', 'anadolu rock', 'arab electronic', 'arabesk',
       'armenian hip hop', 'armenian pop',
       ...
       'turkish folk', 'turkish hip hop', 'turkish pop', 'turkish rock',
       'turkish trap', 'uk dancehall', 'urbano espanol', 'vapor twitch',
       'violin', 'york indie'],
      dtype='object', length=141)

Create a new Dataframe with all the genres to be used for the recommendation later.

In [7]:
pred_df = pd.DataFrame(columns=genrelist)
pred_df

,clubs,acid techno,afro house,alternative dance,alternative hip hop,anadolu rock,arab electronic,arabesk,armenian hip hop,armenian pop,...,turkish folk,turkish hip hop,turkish pop,turkish rock,turkish trap,uk dancehall,urbano espanol,vapor twitch,violin,york indie


## Create the model for regression using Randomforest
Using the Onehotencoded event data from the 'ev_ohe' DataFrame, and the club names from 'ev' Dataframe we can create X and y

In [35]:
clf=RandomForestClassifier(max_depth=10 , random_state=42, bootstrap=True  )

In [9]:
X = ev_ohe.drop('clubs', axis=1)
y = ev_ohe['clubs']

In [43]:
clf.fit(X, y)

RandomForestClassifier(max_depth=5, random_state=42)

## Testing with 1 club to see if it is predicting correct

In [44]:
test=X.iloc[0]   #40secondsclub
test=np.array(test).reshape(1,-1)

In [45]:
clf.predict(test)


/home/nathan/anaconda3/envs/party/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([' 40seconds Club'], dtype=object)

In [48]:
test1=X.iloc[59]   #Yaam
test1=np.array(test1).reshape(1,-1)

In [49]:
clf.predict(test1)


/home/nathan/anaconda3/envs/party/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([' YAAM (club)'], dtype=object)

## Model seems to predict given the correct data for a specific club
Lets try with some of the user data from spotify

In [50]:
recently_played=pd.read_csv('recently_played.csv')

In [53]:
recently_played.head(20)

,artist_id,artist,genre,genre2,genre3,genre4,genre5,genre6,genre7,genre8,genre9,genre10
0,23fqKkggKUBHNkbKtXEls4,Kygo,'edm','pop','pop dance','tropical house',NaN,NaN,NaN,NaN,NaN,NaN
1,3NPpFNZtSTHheNBaWC82rB,X Ambassadors,'modern alternative rock','modern rock','pop','pop rap','pop rock','rock','stomp pop',NaN,NaN,NaN
2,6TIYQ3jFPwQSRmorSezPxX,Machine Gun Kelly,'ohio hip hop',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3NPpFNZtSTHheNBaWC82rB,X Ambassadors,'modern alternative rock','modern rock','pop','pop rap','pop rock','rock','stomp pop',NaN,NaN,NaN
4,3NPpFNZtSTHheNBaWC82rB,X Ambassadors,'modern alternative rock','modern rock','pop','pop rap','pop rock','rock','stomp pop',NaN,NaN,NaN
5,4WN5naL3ofxrVBgFpguzKo,Rudimental,'dance pop','edm','house','pop','pop dance','tropical house','uk dance','uk funky','uk pop',NaN
6,5KKpBU5eC2tJDzf0wmlRp2,RAYE,'dance pop','pop','pop dance','post-teen pop','tropical house','uk contemporary r&b','uk pop',NaN,NaN,NaN
7,4WN5naL3ofxrVBgFpguzKo,Rudimental,'dance pop','edm','house','pop','pop dance','tropical house','uk dance','uk funky','uk pop',NaN
8,2vf4pRsEY6LpL5tKmqWb64,Elderbrook,'house','tropical house',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,4WN5naL3ofxrVBgFpguzKo,Rudimental,'dance pop','edm','house','pop','pop dance','tropical house','uk dance','uk funky','uk pop',NaN


Create a query to use from playlist songs 13:20 all electronic music

In [54]:
query=recently_played[13:20]
query

,artist_id,artist,genre,genre2,genre3,genre4,genre5,genre6,genre7,genre8,genre9,genre10
13,0iTjLBepeGaLgZS18kxgRq,Oliver Schories,'deep euro house','hamburg electronic','minimal techno','tech house',NaN,NaN,NaN,NaN,NaN,NaN
14,0flK7a9sS4TEcCYVNUaiXU,Guille Placencia,'funky tech house','latin tech house',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,3DkvwOqU1uAEt4LGs0EAOv,P.A.C.O.,'deep techno',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,3dds88EVSeBgkPCJ47ZzRZ,Shed,'ambient techno','dub techno','float house','intelligent dance music','microhouse','minimal dub','minimal techno','outsider house','techno','uk bass'
17,3dds88EVSeBgkPCJ47ZzRZ,Shed,'ambient techno','dub techno','float house','intelligent dance music','microhouse','minimal dub','minimal techno','outsider house','techno','uk bass'
18,6EROUCmvQtSuBUkZ9nNOOO,Amotik,'minimal dub',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,0f4ne0cd4tbiZCLzpb1QoX,Developer,'minimal dub',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
query['all'] = query[query.columns[2:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)
query

/tmp/ipykernel_28574/2463477683.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query['all'] = query[query.columns[2:]].apply(


,artist_id,artist,genre,genre2,genre3,genre4,genre5,genre6,genre7,genre8,genre9,genre10,all
13,0iTjLBepeGaLgZS18kxgRq,Oliver Schories,'deep euro house','hamburg electronic','minimal techno','tech house',NaN,NaN,NaN,NaN,NaN,NaN,"'deep euro house', 'hamburg electronic', 'mini..."
14,0flK7a9sS4TEcCYVNUaiXU,Guille Placencia,'funky tech house','latin tech house',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"'funky tech house', 'latin tech house'"
15,3DkvwOqU1uAEt4LGs0EAOv,P.A.C.O.,'deep techno',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'deep techno'
16,3dds88EVSeBgkPCJ47ZzRZ,Shed,'ambient techno','dub techno','float house','intelligent dance music','microhouse','minimal dub','minimal techno','outsider house','techno','uk bass',"'ambient techno', 'dub techno', 'float house',..."
17,3dds88EVSeBgkPCJ47ZzRZ,Shed,'ambient techno','dub techno','float house','intelligent dance music','microhouse','minimal dub','minimal techno','outsider house','techno','uk bass',"'ambient techno', 'dub techno', 'float house',..."
18,6EROUCmvQtSuBUkZ9nNOOO,Amotik,'minimal dub',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'minimal dub'
19,0f4ne0cd4tbiZCLzpb1QoX,Developer,'minimal dub',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'minimal dub'


In [58]:
q=query['all']
q= q.str.cat(sep=", ")
q=q.split(",")
q = [x[1:-1] for x in q]
q

['deep euro house',
 "'hamburg electronic",
 "'minimal techno",
 "'tech house",
 "'funky tech house",
 "'latin tech house",
 "'deep techno",
 "'ambient techno",
 "'dub techno",
 "'float house",
 "'intelligent dance music",
 "'microhouse",
 "'minimal dub",
 "'minimal techno",
 "'outsider house",
 "'techno",
 "'uk bass",
 "'ambient techno",
 "'dub techno",
 "'float house",
 "'intelligent dance music",
 "'microhouse",
 "'minimal dub",
 "'minimal techno",
 "'outsider house",
 "'techno",
 "'uk bass",
 "'minimal dub",
 "'minimal dub"]

In [73]:
user_data = pd.DataFrame(q)
user_data.columns=["genre"]
user_data_str = user_data['genre'].str.cat(sep=', ')
user_data_str

"deep euro house, 'hamburg electronic, 'minimal techno, 'tech house, 'funky tech house, 'latin tech house, 'deep techno, 'ambient techno, 'dub techno, 'float house, 'intelligent dance music, 'microhouse, 'minimal dub, 'minimal techno, 'outsider house, 'techno, 'uk bass, 'ambient techno, 'dub techno, 'float house, 'intelligent dance music, 'microhouse, 'minimal dub, 'minimal techno, 'outsider house, 'techno, 'uk bass, 'minimal dub, 'minimal dub"

In [74]:
result = []

for columname in pred_df.columns:
    if columname in user_data_str:
        result.append(1)
    else:
        result.append(0)

In [75]:
pred_df.loc[len(pred_df.index)] = result

In [76]:
pred_df_1=pred_df.drop('clubs', axis=1)
pred_df_1


,acid techno,afro house,alternative dance,alternative hip hop,anadolu rock,arab electronic,arabesk,armenian hip hop,armenian pop,aussietronica,...,turkish folk,turkish hip hop,turkish pop,turkish rock,turkish trap,uk dancehall,urbano espanol,vapor twitch,violin,york indie
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
pred_df.value_counts()

clubs  acid techno  afro house  alternative dance  alternative hip hop  anadolu rock  arab electronic  arabesk  armenian hip hop  armenian pop  aussietronica  australian dance  australian electropop  australian house  australian pop  australian techno  auteur-compositeur-interprete quebecois  belgian hip hop  belgian indie  belgian pop  bergen indie  bleep techno  bluegrass fiddle  britpop  canadian electronic  chanson  chanson quebecois  classic french pop  classical  classical era  classical tenor  compositional ambient  conscious hip hop  coverchill  czsk hip hop  dance pop  dance rock  dancehall chileno  danish electronic  danish electropop  danish techno  deep soul house  deep tropical house  detroit house  disco house  downtempo  dub techno  dutch edm  east coast hip hop  edm  electro house  electro-pop francais  electronic trap  electronica  electropop  emo rap  ethnotronica  eurovision  experimental house  experimental techno  float house  focus beats  fotbollslatar  frankfurt 

In [69]:
clf.predict(pred_df_1)

array([' 40seconds Club'], dtype=object)

## model development

In [26]:
clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 5,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [27]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [82]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X, y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


ValueError: n_splits=3 cannot be greater than the number of members in each class.